# Generation of table ideals

This file contains the generating functions for table and non-table ideals, and some operation functions on tables.

In [ ]:
import random
import numpy as np

This code generates a single table for table ideals.
Conditions are integers $s<n$ and a maximum size for an entry in the table

In [ ]:
def table(s, n, maxent):
    '''
    s is the numer of relations, n is the number of columns/variables, 
    and maxent is the maximum size for a random entry in the table. 
    '''
    T = []
    for i in range(s+1):
        t = []
        for j in range(n):
            if j < i and i < s:
                t.append(0)
            elif i < s:
                t.append(random.randint(0, maxent))
            elif i >= s:
                t.append(0)
        T.append(t)
    # bounding the ii entries in the table
    r = s-1
    while r >- 1:
        B = T[r+1][r+1]
        for j in range(r+1,n):
            B = B+T[r][j]
        T[r][r] = random.randint(0,B)
        r = r-1
    
    #this counts the column sums
    CS = []
    for i in range(n):
        A = 0
        for j in range(s):
            A = A+T[j][i]
        CS.append(A)
    #we define the row of d_i
    D = []
    for i in range(n):
        if i < s:
            d = T[i+1][i+1]
            for k in range(i):
                d = d+T[k][i]
            for t in range(i+1,n):
                d = d+T[i][t]
            D.append(d)
        else:
            D.append(random.randint(CS[i],maxent*(n+s)))
    T.insert(0,D)
    return T

def deduct(L, K):
    N = [L[i]-K[i] for i in range(len(L))]
    return N


def table_ideal(T):
    '''
    This function takes in a table and generates a table ideal.
    The input is a table and the output is a monomial ideal given by exponent vectors
    '''
    I = []
    s = len(T)-2 #-1 if the last row of zeros is deleted
    n = len(T[0])
    M = T.copy()
    mono = [0 for j in range(n)]
    
    for i in range(s+1):
        if i > 0:
            M[0] = deduct(M[0], M[i])
            mono[i-1] = M[0][i-1]
        
        for j in range(i,n):
            m = mono.copy()
            m[j] = M[0][j]
            I.append(m)
    return I

In [ ]:
# The following functions are used to unifromise the shape of the exponent vectors

def merge(L, is_table=True):
    M = []
    for x in L:
        M = M + list(x)
    if is_table:
        M.append(1) #label, this is 1 for table ideals and 0 for others
    else:
        M.append(0)
    return M


def filler(L, n, N):
    '''
    We want all the exponent vectors to be the same length
    L is the list of monomial exponent vectors
    n is number of variables
    '''
    k = len(L)
    v = [0 for i in range(n)]
    M = L.copy()
    for i in range(N-k):
        M.insert(0, v)
    return M

## Example: Table ideals, 10 variables, monomial degrees up to 7

In [ ]:
s = 7
n = 10
maxent =  1000
N = int(1030/10) # from the paper

table_ideals = [table_ideal(table(s, n, maxent)) for i in range(10000)]
table_ideals_uniform = [merge(filler(I, n, N)) for I in table_ideals]

In [ ]:
import csv
filename = 'table_exponents_10var.csv'
with open(filename, 'w') as f:
        c = csv.writer(f)
        c.writerows(table_ideals_uniform)
f.close()

# The following code is to generate almost-table ideals

In [ ]:
def tablemod(s, T):
    '''
    This function modifies the table to give almost table ideals
    '''
    t = random.randint(0,s-1)
    w = random.randint(0,t+1)
    k = random.choice([-2,-1,1,2])
    T[w][t] = T[w][t]+k
    return T
    
    
def symnontables(n, max_ent):
    '''
    This function gives symmetric non-table ideals
    '''
    mons = []
    L = [random.randint(0,max_ent) for i in range(n)]
    gen = [0 for i in range(n)]
    gen2 = [0 for i in range(n)]
    s = 0
    for x in L:
        s = s + x
    gen2[n-1] = s
    for x in range(n-1):
        v = gen.copy()
        w = gen2.copy()
        v[x] = L[x+1] + 1
        w[x] = 1
        mons.append(v)
        mons.append(w)
    gen2[n-1] = gen2[n-1] + 1
    mons.append(gen2)
    return mons


def sortswap(T, P):
    '''
    The function generates non-table ideals from a permutation of a table.
    '''
    n1 = len(T) - 1
    n2 = len(T[0])
    tab = np.array(T)
    subs = tab[0]
    subl = []
    for i in range(n1-1):
        subs = subs - tab[i+1]
        subl.append(subs.tolist())
    
    marker = 0
    for x in subl:
        if 0 in x:
            marker = 1
    
    if marker == 1:
        return []
    else:
        ideals = []
        #possible premutations for T
        perm = []
        
        ha = T[1][0]
        for j in T[0]:
            if j < ha:
                perm.append(T[0].index(j))
        L = table_ideal(T)
            
        mon = [0 for i in range(n2)]
        for x in perm:
            for y in P:
                ide = []
                for z in L:
                    z1 = z.copy()
                    z1[0] = z[x]
                    z1[x] = z[0]
                    mon2 = mon.copy()
                    mon2[0] = z[x]
                    for i in range(n2-1):
                        mon2[i+1] = z1[y[i]+1]
                    ide.append(mon2)
                ideals.append(ide)
        return ideals
    
    
def nontable2(n, num):
    '''
    Computes the desired amount of ideals 
    '''
    nontab = []
    for i in range(n-1):
        count = 0
        while count < round(num/n):
            K = sortswap(table(i+1, n, 1000), np.permutation(n-1))
            if K != []:
                for x in K:
                    nontab.append(merge(filler(sorted(x, key=itemgetter(slice(0, None))),n)), is_table=False)
                    count=count+1
    return nontab

# We produce non-table ideals with the help of Macaulay2

We first generate monomials of certain form. Then we use these to generate non-table ideals using Macaulay2

In [ ]:
import random
import itertools

In [ ]:
# Call Macaulay2, which we assume the user has installed locally
m2 = Macaulay2(command='/usr/local/bin/M2')

In [ ]:
def python_to_macaulay(I, vars):
    '''
    Converts python code to macaulay2 code
    I ideal as a python string
    '''
    J = m2('R = QQ[' + vars + ']; I = inverseSystem(matrix{{'+ I + '}}, DividedPowers = >true); J=ideal(leadTerm(I)); toString J')
    return(J)
    
def ideal_to_exponent(J, vars):
    vars_sep = vars.split(',')
    n = len(vars_sep)
    P = PolynomialRing(QQ, n, names=vars_sep)
    I = str(J)
    if I[len(I)-1] == ')':
        I = I[6:len(I)-1]
    else:
        I = I[6:len(I)-2]
    I_vec = I.split(",")
    f = [P(mon) for mon in I_vec]
    exp = [mon.exponents()[0] for mon in f]
    return(exp)

In [ ]:
dic2 = {1:[[1]]}
def parti(n, k):
    '''
    Partition function. n is the number to be partitioned and k is max length we can have
    '''
    partitions = [[n]]
    for i in range(1, n):
        if i in dic2:
            w = dic2[i].copy()
        if i not in dic2:
            w = parti(i, i)
            dic2[i] = w.copy()
        rem = []
        if n-i < i:
            for x in w:
                if x[0] > n-i:
                    rem.append(x)
        for y in rem:
            w.remove(y)
        for x in w:
            partitions.append([n-i] + x)
    d = []
    for x in partitions: 
        if len(x) <= k:
            d.append(x)
    return d


def varchoice(n, k):
    '''
    This function generates collection of monomials in degree d
    '''
    v = [i for i in range(n)]
    L = list(itertools.combinations(v, k))
    r = random.randint(0, len(L)-1)
    return L[r]


def totext(t,n):
    '''
    Input tuple and number of var, denote x1,x2,x3,...
    '''
    mono = ''
    for i in range(n):
        if i == 0:
            mono = mono + 'x' + str(i+1) + '^'+str(t[i])
        else:
            mono = mono + '*x' + str(i+1) + '^'+str(t[i])
    return mono


def forms(n, d):
    forms = set()
    v = [0 for i in range(n)]
    P = parti(d, n)
    s = random.randint(2, 10)
    I = [random.randint(0, len(P)-1) for i in range(s)]
    
    for x in I:
        k = len(P[x])
        random.shuffle(P[x])
        V = varchoice(n,k)
        mon = v.copy()
        for y in V:
            mon[y] = P[x][V.index(y)] 
        mon2 = tuple(mon)
        forms.add(mon2)
    m2input = ''
    for y in forms:
        m2input = m2input + totext(y,n) + ', '
    m2i = m2input.rstrip()
    return m2i.rstrip(',')

## Example: Non-table ideals, 10 variables, monomial degrees up to 7

In [ ]:
n_vars = 10
deg_bound = 7
polys = [forms(n_vars,random.randint(2,deg_bound-1)) for d in range(20000)]
ideals_random = [python_to_macaulay(f, 'x1,x2,x3,x4,x5,x6,x7,x8,x9,x10') for f in polys]

In [ ]:
expo_ideals = [ideal_to_exponent(J, 'x1,x2,x3,x4,x5,x6,x7,x8,x9,x10') for J in ideals_random]
expo_length = max([len(expo_ideals[i]) for i in range(len(expo_ideals))])

In [ ]:
import csv
filename = 'non_table_exponents_10vars_deg7.csv'
with open(filename, 'w') as f:
        c = csv.writer(f)
        c.writerows(expo_ideals)
f.close()